Aprendizado de Máquina - 2024/3

Aluno: Álvaro de Carvalho Alves

# Questão 1

Reinforcement Learning, Conceitos

Explique a diferença entre Value iteration, Policy iteration e Q-learning. Para facilitar a explicação, mostre as equações usadas para cada caso.

## **Resposta:**



### Value Iteration: 

Calcula o valor ótimo de cada estado até convergir. A política ótima é então derivada desses valores ótimos.

Equação:

$$ V(s) = \max_a \left \{ \sum_{s'} P(s'|s,a) \cdot \left[ R(s,a,s') + \gamma \cdot V(s') \right] \right \} $$

Onde:

$V(s)$ é o valor do estado $s$

$a$ é uma ação possível

$s'$ é um possível estado sucessor

$P(s'|s,a)$ é a probabilidade de transição do estado $s$ para $s'$ ao tomar a ação $a$

$R(s,a,s')$ é a recompensa recebida ao ir de $s$ para $s'$ com a ação $a$

$\gamma$ é o fator de desconto (quão importante são as recompensas futuras)


### Policy Iteration

Alterna entre a avaliação da política atual e a melhoria da política com base nessa avaliação.

Equações:

- Policy Evaluation:

$$ V_\pi(s) = \sum_{s'} P(s'|s,\pi(s)) \cdot \left[ R(s,\pi(s),s') + \gamma * V_\pi(s') \right] $$


- Policy Improvement:

$$ \pi'(s) = argmax_a \left \{ \sum_{s'} P(s'|s,a) \cdot \left [ R(s,a,s') + \gamma \cdot V_\pi(s') \right ] \right \} $$

Onde:

$V_\pi(s)$ é o valor do estado $s$ sob a política $\pi$

$\pi(s)$ é a ação que a política $\pi$ dita no estado $s$

$\pi'(s)$ é a ação da política melhorada no estado $s$

### Q-Learning

Aprende diretamente a função $Q$, que representa o valor de tomar uma ação em um determinado estado. Não precisa de um modelo do ambiente.

Equação:

$$ Q(s,a) = Q(s,a) + \alpha \cdot \left[ R(s,a,s') + \gamma \cdot \max_{a'} Q(s',a') - Q(s,a) \right] $$

Onde:

$Q(s,a)$ é o valor de tomar a ação $a$ no estado $s$

$\alpha$ é a taxa de aprendizado (quão rápido o algoritmo aprende com novas informações)

$\max_{a'} Q(s',a')$ é o valor estimado da melhor ação no próximo estado $s'$

# Questão 2
Otimização de um Sistema de Fila.

Considere um sistema de fila onde clientes chegam e são atendidos em intervalos de tempo discretos (a cada slot) de tempo. O sistema é definido a seguir:

1. Chegada de Clientes:
    - No final de cada intervalo de tempo: 0, 2 ou 4 clientes chegam com probabilidades $p_0$, $p_2$ e $p_4$, respectivamente $(p_0 + p_2 + p_4 = 1)$.

  
2. Atendimento:
    - No início de cada intervalo de tempo: Se houver $S$ servidores e $C$ clientes, o sistema atende $min(S, C)$ clientes.
  
    - Clientes restantes permanecem no sistemas (ficam para o próximo intervalo de tempo).

3. Restrições do Sistema (para facilitar):
    - Um máximo de 8 clientes pode estar no sistema em qualquer momento.
    - O sistema inicia no estado inicial $(C, S) = (0,0)$, isto é, sem usuários e sem nenhum servidor.
    
4. Ações:
    - No início de cada intervalo de tempo, pode-se:
        - **-1:** Remover 1 servidor.
        - **0:** Manter o número atual de servidores.
        - **+1:** Adicionar 1 servidor.
    - O número de servidores é limitado entre 1 e 3.

5. Recompensas e Custos:
    - **Ganhos:** $T$ por cliente atendido.
    - **Custo do Servidor:** $R_s$ por servidor por intervalo de tempo.
    - **Penalidade de Fila:** $R_q$ por slot, quando houver mais que 4 clientes no sistema.
    - **Penalidade de Ociosidade:** $R_0$ por cada servidor não utilizado por intervalo de tempo.
6. Objetivo:
    - Projetar uma política $\pi(s)$ que maximize a recompensa esperada:
    - Você deve escolher um valor $\gamma$ para o fator de desconto.
  
7. Você pode escolher entre value iteration, policy iteration, etc.

## **RESPOSTA:**

In [1]:
# Configurações do problema
MAX_CLIENTS = 8
MIN_SERVERS = 1
MAX_SERVERS = 3

ARRIVAL_PROBS = {0: 0.4, 2: 0.2, 4: 0.4}  # p_0, p_2, p_4

T = 10  # Ganho por cliente atendido
R_S = 5  # Custo por servidor por intervalo
R_Q = 10  # Penalidade por mais de 4 clientes na fila
R_0 = 2  # Penalidade por servidor ocioso

GAMMA = 0.9  # Fator de desconto
EPSILON = 1e-3  # Critério de convergência

### Componentes do MDP
Para o MDP, precisamos dos estados, ações, probabilidade de transições e recompensas.



#### Estados:
Um estado do sistema é descrito por:

$ s = (C, S) $

Onde $C$ é o número de clientes no sistema ($0 \leq C \leq 8$) e $S$ é número de servidores no sistema ($1 \leq S \leq 3$).

O espaço de estados tem tamanho $9 \times 3 = 27$.

In [2]:
states = [(C, S) for C in range(MAX_CLIENTS + 1) for S in range(MIN_SERVERS, MAX_SERVERS + 1)]

#### Ações:
As ações possíveis são:

$ a \in \{-1, 0, +1\} $

Que representam:

- **Remover (-1):** Decrementar o número de servidores (mínimo 1).

- **Manter (0):** Manter o número atual de servidores.

- **Adicionar (+1):** Incrementar o número de servidores (até 3).

In [3]:
actions = [-1, 0, +1]

#### Transições:
Os próximos estados dependem de:

1. **Número de chegadas:** 0, 2, ou 4 clientes com probabilidades $p_0, p_2, p_4$.

2. **Capacidade do sistema:** Número de clientes atendidos por $S$ servidores ($\min(S, C)$).

3. **Fila:** Os clientes não atendidos permanecem no sistema até o próximo intervalo.

Dessa forma, dado um estado atual $(C, S)$ e uma ação $action$, um novo estado $(new\_C, new\_S)$ é computado por:

$$ new\_S = \max(1, \min(3, S + action)) $$
$$ served\_clients = \min(S, C)$$
$$ remaining\_clients = C - served\_clients $$
$$ next\_C = \min(8, remaining\_clients + arrivals) $$

Para cada chegada $arrivals \in {0, 2, 4}$.

In [4]:
import numpy as np

def get_next_states_and_probabilities(C, S, action):
    """ Retorna os estados possíveis e suas probabilidades após a ação."""
    next_states = {}
    new_S = max(MIN_SERVERS, min(MAX_SERVERS, S + action))  # Limita o número de servidores

    for arrivals, prob in ARRIVAL_PROBS.items():
        served_clients = min(S, C)  # Atende no máximo min(servidores, clientes)
        remaining_clients = C - served_clients
        next_C = min(MAX_CLIENTS, remaining_clients + arrivals)  # Aplica limite da fila

        if (next_C, new_S) not in next_states:
            next_states[(next_C, new_S)] = 0
        next_states[(next_C, new_S)] += prob

    return next_states

#### Recompensas:
A recompensa depende de:

1. **Clientes atendidos:** $T \times \text{número de clientes atendidos}$.

2. **Custo dos servidores:** $R_s \times S$.

3. **Penalidade de fila cheia:** $R_q$ se $C > 4$.

4. **Penalidade por ociosidade:** $R_0 \times (S - \min(S, C))$.

A função de recompensa é:
$$ R(s, a, s') = T \times \text{clientes atendidos} - R_s \times S - R_q \cdot \mathbb{1}(C > 4) - R_0 \cdot (S - \text{clientes atendidos}) $$

In [5]:
def get_reward(C, S, action, next_C, next_S):
    """ Calcula a recompensa de transição para um estado específico."""
    served_clients = min(S, C)  # Clientes atendidos
    idle_servers = S - served_clients

    reward = T * served_clients  # Ganhos por clientes atendidos
    reward -= R_S * S           # Custo por servidores

    if next_C > 4:
        reward -= R_Q           # Penalidade de fila cheia

    reward -= R_0 * idle_servers  # Penalidade por servidores ociosos

    return reward

### 2.1. Value Iteration:

1. Inicializar $V(s)$ arbitrariamente (aqui inicializo como 0).

2. Para cada iteração $k$, atualize:
$$ V_{k+1}(s) = \max_a \sum_{s'} P(s'|s, a) \left[R(s, a, s') + \gamma V_k(s')\right] $$

3. Parar quando convergir:
$$ \max_s |V_{k+1}(s) - V_k(s)| < \epsilon $$

4. Política resultante:
$$ \pi(s) = \arg\max_a \sum_{s'} P(s'|s, a) \left[R(s, a, s') + \gamma V(s')\right] $$

In [6]:
# Value Iteration
V = {state: 0 for state in states}  # Inicialização dos valores dos estados
policy = {state: 0 for state in states}  # Política inicial

In [7]:
while True:
    delta = 0  # Diferença máxima entre iterações
    new_V = V.copy()

    for state in states:
        C, S = state
        best_value = float('-inf')
        best_action = 0

        for action in actions:
            # Calcular o valor esperado da ação
            value = 0
            next_states = get_next_states_and_probabilities(C, S, action)

            for (next_C, next_S), prob in next_states.items():
                reward = get_reward(C, S, action, next_C, next_S)
                value += prob * (reward + GAMMA * V[(next_C, next_S)])

            # Atualiza a melhor ação
            if value > best_value:
                best_value = value
                best_action = action

        # Atualiza o valor do estado
        new_V[state] = best_value
        policy[state] = best_action

        # Calcula a mudança máxima
        delta = max(delta, abs(new_V[state] - V[state]))

    V = new_V

    # Critério de convergência
    if delta < EPSILON:
        break

# Exibir resultados finais
print("Política Ótima:")
for state in sorted(policy):
    print(f"Estado {state}: Ação {policy[state]}")

print("\nValores dos Estados:")
for state in sorted(V):
    print(f"Estado {state}: Valor {V[state]:.2f}")

Política Ótima:
Estado (0, 1): Ação 1
Estado (0, 2): Ação 0
Estado (0, 3): Ação -1
Estado (1, 1): Ação 1
Estado (1, 2): Ação 0
Estado (1, 3): Ação -1
Estado (2, 1): Ação -1
Estado (2, 2): Ação 0
Estado (2, 3): Ação -1
Estado (3, 1): Ação 1
Estado (3, 2): Ação -1
Estado (3, 3): Ação -1
Estado (4, 1): Ação 1
Estado (4, 2): Ação 0
Estado (4, 3): Ação -1
Estado (5, 1): Ação 1
Estado (5, 2): Ação 1
Estado (5, 3): Ação -1
Estado (6, 1): Ação 1
Estado (6, 2): Ação 1
Estado (6, 3): Ação 0
Estado (7, 1): Ação 1
Estado (7, 2): Ação 1
Estado (7, 3): Ação 0
Estado (8, 1): Ação 1
Estado (8, 2): Ação 1
Estado (8, 3): Ação 0

Valores dos Estados:
Estado (0, 1): Valor 21.33
Estado (0, 2): Valor 14.33
Estado (0, 3): Valor 7.33
Estado (1, 1): Valor 33.33
Estado (1, 2): Valor 26.33
Estado (1, 3): Valor 19.33
Estado (2, 1): Valor 34.86
Estado (2, 2): Valor 38.33
Estado (2, 3): Valor 31.33
Estado (3, 1): Valor 40.20
Estado (3, 2): Valor 39.86
Estado (3, 3): Valor 43.33
Estado (4, 1): Valor 38.47
Estado (4,

### Simulação

In [10]:
import random

def simulate(policy, steps=10):
    """ Simula as transições no sistema de filas usando a política ótima."""
    state = (0, MIN_SERVERS)  # Estado inicial
    history = []

    for _ in range(steps):
        C, S = state
        action = policy[state]  # Ação baseada na política

        # Determinar próximo estado
        next_states = get_next_states_and_probabilities(C, S, action)
        next_state = random.choices(list(next_states.keys()), weights=next_states.values())[0]

        # Calcular recompensa
        reward = get_reward(C, S, action, *next_state)

        # Registrar histórico
        history.append((state, action, reward, next_state))

        # Atualizar estado
        state = next_state

    return history

# Executar simulação
simulation_history = simulate(policy, steps=15)

print("\nSimulação de Transições:")
for i, (state, action, reward, next_state) in enumerate(simulation_history):
    print(f"Passo {i+1}: Estado {state} -> Ação {action} -> Recompensa {reward:.2f} -> Próximo Estado {next_state}")


Simulação de Transições:
Passo 1: Estado (0, 1) -> Ação 1 -> Recompensa -7.00 -> Próximo Estado (4, 2)
Passo 2: Estado (4, 2) -> Ação 0 -> Recompensa 10.00 -> Próximo Estado (2, 2)
Passo 3: Estado (2, 2) -> Ação 0 -> Recompensa 10.00 -> Próximo Estado (4, 2)
Passo 4: Estado (4, 2) -> Ação 0 -> Recompensa 10.00 -> Próximo Estado (4, 2)
Passo 5: Estado (4, 2) -> Ação 0 -> Recompensa 0.00 -> Próximo Estado (6, 2)
Passo 6: Estado (6, 2) -> Ação 1 -> Recompensa 0.00 -> Próximo Estado (8, 3)
Passo 7: Estado (8, 3) -> Ação 0 -> Recompensa 5.00 -> Próximo Estado (8, 3)
Passo 8: Estado (8, 3) -> Ação 0 -> Recompensa 5.00 -> Próximo Estado (8, 3)
Passo 9: Estado (8, 3) -> Ação 0 -> Recompensa 5.00 -> Próximo Estado (8, 3)
Passo 10: Estado (8, 3) -> Ação 0 -> Recompensa 5.00 -> Próximo Estado (7, 3)
Passo 11: Estado (7, 3) -> Ação 0 -> Recompensa 5.00 -> Próximo Estado (8, 3)
Passo 12: Estado (8, 3) -> Ação 0 -> Recompensa 5.00 -> Próximo Estado (7, 3)
Passo 13: Estado (7, 3) -> Ação 0 -> Recomp